In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle




In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df= df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [18]:
le_salary= LabelEncoder()
df['Gender'] = le_salary.fit_transform(df["Gender"])

In [9]:
ohe = OneHotEncoder()
geo_encoded= ohe.fit_transform(df[['Geography']]).toarray()

geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe.get_feature_names_out(['Geography']))


In [10]:
df=pd.concat([df.drop("Geography",axis=1), geo_encoded_df], axis=1)

In [12]:
X=df.drop('EstimatedSalary', axis=1)
y=df['EstimatedSalary']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
scaler= StandardScaler()
X_trained = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
# saving the pickle file
#save encoder data 
pickle.dump(ohe, open('ohe.pkl', 'wb'))
#save label encoder data
pickle.dump(le, open('le_salary.pkl', 'wb'))
#save scaler data
pickle.dump(scaler, open('scaler.pkl', 'wb'))

In [22]:
X_train.shape[1],

(12,)

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [25]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression
    ]
)

c:\personal\Churn_ANN\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [27]:
#compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,373 (40.52 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,916 (27.02 KB)

In [32]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16,validation_data=(X_test,y_test), validation_split=0.2, callbacks=[EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True), TensorBoard(log_dir='regressionlogs/fit')])

Epoch 1/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3513806592.0000 - mae: 50785.1797 - val_loss: 12950033408.0000 - val_mae: 98226.7812
Epoch 2/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3532287488.0000 - mae: 50872.5273 - val_loss: 12951593984.0000 - val_mae: 98234.6484
Epoch 3/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3516972288.0000 - mae: 50762.0938 - val_loss: 12951536640.0000 - val_mae: 98234.3203
Epoch 4/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3518956288.0000 - mae: 50837.3516 - val_loss: 12950505472.0000 - val_mae: 98229.0938
Epoch 5/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3514597632.0000 - mae: 50827.0898 - val_loss: 12945950720.0000 - val_mae: 98205.9922
Epoch 6/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3509071360.0000 - mae: 50727.3633 - val_loss: 12947329024.0000 - val_mae: 98212.9609
Epoch 7/100
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 3506620160.0000 - mae: 50756.8789 - val_loss: 12943958016.0000 

In [33]:
# evaluate the model
%load_ext tensorboard

In [36]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 22064), started 0:01:31 ago. (Use '!kill 22064' to kill it.)

In [38]:
test_loss, test_mae=model.evaluate(X_test,y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 12833754112.0000 - mae: 97633.6797  
Test Loss: 12833754112.0, Test MAE: 97633.6796875


In [39]:
model.save('salary_regression_model.h5')